### DATA CLEANING

In [2]:
import pandas as pd
from thefuzz import process   

def clean_text(string, choices):
    try:
        string = string.lower()
        potential_matches = process.extract(string, choices, limit=2)
        for match in potential_matches:
            if match[1] >= 75:
                return match[0]
        return string
    except AttributeError:
        return string


set up the variables that will be passed as parameters on the read_excel function

In [3]:

valenzuela_filename = "./raw_data/Valenzuela_Data.xlsx"
zamboanga_filename = "./raw_data/Zamboanga_Data.xlsx"

to_col_names = ['code',
             'sex',
             'age',
             'grade',
             'class_shift',
             'respondent',
             'modes_of_transport',
             'main_mode',
             '-cycle_helmet',
             '-cycle_number',
             'front_seat',
             'seatbelt',
             'travel_time',
             'travel_time>30',
             'companion_bool',
             'companion',
             'baranggay',
             'incident_bool',
             'incident_location',
             'incident_specific_location',
             'incident_vehicle_bool',
             'incident_vehicle',
             'close_call_bool',
             'close_call_location',
             'close_call_specific_location',
             'close_call_vehicle_bool',
             'close_call_vehicle',
             ]

from_col_names = ['code',
                'sex',
                'age',
                'grade',
                'class_shift',
                'respondent',
                'modes_of_transport',
                'main_mode',
                '-cycle_helmet',
                '-cycle_number',
                'front_seat',
                'seatbelt',
                'travel_time',
                'travel_time>30',
                'companion_bool',
                'companion',
                'location_after_school_bool',
                'location_after_school',
                'baranggay',
                'incident_bool',
                'incident_location',
                'incident_specific_location',
                'incident_vehicle_bool',
                'incident_vehicle',
                'close_call_bool',
                'close_call_location',
                'close_call_specific_location',
                'close_call_vehicle_bool',
                'close_call_vehicle'
                ]

dtype_cols = {'-cycle_helmet': 'boolean',
             'front_seat': 'boolean',
             'companion_bool': 'boolean',
             'incident_bool': 'boolean',
             'close_call_bool': 'boolean',
             'incident_vehicle_bool': 'boolean',
             'main_mode': 'category',
             '-cycle_number': 'category',
             'travel_time': 'category',
             'companion': 'category',
             'incident_location': 'category',
             'incident_vehicle': 'category',
             'close_call_location': 'category',
             'close_call_vehicle': 'category'
             }


to_school_cols = "C:E,G:I,K:AE"
from_school_cols = "C:E,G:I,AF:BB"

bool_dict = {'Yes': True, 'No': False, 'I cannot remember anymore (Hindi na maalala)': False}

pd.set_option('display.max_columns', None)

## __Valenzuela: To School__ dataset

bool columns do not include 'seatbelt' since I want to observe the distinction between seatbelt and childseat usage

In [4]:
valenzuela_to_school = pd.read_excel(valenzuela_filename,
                                    usecols=to_school_cols,
                                    names=to_col_names,
                                    dtype=dtype_cols,
                                    true_values=['Yes'],
                                    false_values=['No'])

### Cleaning the data in `valenzuela_to_school` dataframe

Columns needed to be cleaned are 

* 'close_call_vehicle_bool'
* 'baranggay'
* 'companion' 

##### Cleaning `valenzuela_to_school['close_call_vehicle_bool']`

After cleaning, the dtype of the column should be 'boolean' since the column can contain NaN values

In [5]:
valenzuela_to_school['close_call_vehicle_bool'] = valenzuela_to_school['close_call_vehicle_bool'].replace(bool_dict).astype('boolean')
valenzuela_to_school['close_call_vehicle_bool'].unique()


<BooleanArray>
[<NA>, True, False]
Length: 3, dtype: boolean

##### Cleaning `valenzuela_to_school['baranggay']`

After cleaning, the column should not contain misspelled baranggay names. 

In [6]:
bgy_list1 = ['assumption ville', 
            'balangkas', 
            'canumay', 
            'coloong', 
            'gen. t. de leon', 
            'karuhatan', 
            'lawa', 
            'lawang bato', 
            'lingunan', 
            'malinta', 
            'marulas', 
            'marulas', 
            'meycauayan', 
            'pinagbayanan', 
            'quezon city', 
            'ugong', 
            'viente reales'
            ]

bgy_dict1 = {'brgy 176 bagong silang':'caloocan', 
            'tampoy 2':'marulas', 
            'p.gregorio st.':'lingunan', 
            'assumption ville':'lingunan',
            'pinagbayanan':'lingunan',
            '688 t santaigo':'viente reales',
            'pacheco village gml':'malinta',
            'lawa':'meycauayan, bulacan',
            'ciudad grande ph2':'meycauayan, bulacan',
            'meycauayan':'meycauayan, bulacan',
            'barangay sto.niño, biñan laguna':'binan laguna'
            }

valenzuela_to_school['baranggay'] = valenzuela_to_school['baranggay'].apply(clean_text, args=[bgy_list1])
valenzuela_to_school['baranggay'].replace(bgy_dict1, inplace=True)
valenzuela_to_school['baranggay'].unique()

array(['balangkas', 'caloocan', 'punturin', 'ugong', 'rincon',
       'gen. t. de leon', 'marulas', 'lingunan', 'meycauayan, bulacan',
       'canumay', 'parada', 'viente reales', 'coloong', 'karuhatan',
       'lawang bato', 'quezon city', 'malinta', 'valenzuela ',
       'binan laguna'], dtype=object)

##### Cleaning `valenzuela_to_school['companion']`

Misspelled categories phrases containing additional words are cleaned to have uniform categories.

Furthermore, categories that are similar to each other are grouped. The following merges were done.

* merged `grandmother` and `grandfather` into a single category `grandparent`
* merged `sister` and `brother`, regardless of age, into a single category `sibling`

In [7]:
cmp_list1 = ['auntie',
            'service',
            'guardian',
            'grandmother',
            'grandfather',
            'brother',
            'sister',
            'grandparent',
            'father',
            'mother',
            'friends',
            'schoolmate'
            ]

cmp_cat1 = {'auntie':'adult relative',
            'brother':'sibling',
            'driver':'service',
            'friends':'schoolmate',
            'grandfather':'grandparent',
            'grandmother':'grandparent',
            'kamag anak':'adult relative',
            'kapatid':'sibling',
            'lola':'grandparent',
            'lolo':'grandparent',
            'parents / relatives':'adult relative',
            'tita':'adult relative',
            'tito':'adult relative',
            'uncle tricycle driver':'service',
            'yaya':'guardian'
            }

valenzuela_to_school['companion'] = valenzuela_to_school['companion'].apply(clean_text, args=[cmp_list1]).astype('category')
valenzuela_to_school['companion'] = valenzuela_to_school['companion'].cat.add_categories('none').fillna('none')
valenzuela_to_school['companion'].replace(cmp_cat1, inplace=True)
valenzuela_to_school['companion'].cat.categories



Index(['adult relative', 'sibling', 'cousin', 'father', 'grandparent',
       'guardian', 'mother', 'schoolmate', 'service', 'none'],
      dtype='object')

## __Valenzuela: From School__ dataset

In [8]:
dtype_cols['location_after_school_bool'] = 'boolean'

valenzuela_from_school = pd.read_excel(valenzuela_filename,
                                        usecols=from_school_cols,
                                        names=from_col_names,
                                        dtype=dtype_cols,
                                        true_values=['Yes'],
                                        false_values=['No'])

### Cleaning the data in `valenzuela_from_school` dataframe

Columns needed to be cleaned are 
* 'close_call_vehicle_bool'
* 'baranggay'
* 'location_after_school'
* 'companion' 

##### Cleaning `valenzuela_from_school['close_call_vehicle_bool']`

After cleaning, the dtype of the column should be 'boolean' since the column can contain NaN values

In [9]:
valenzuela_from_school['close_call_vehicle_bool'] = valenzuela_from_school['close_call_vehicle_bool'].replace(bool_dict).astype('boolean')
valenzuela_from_school['close_call_vehicle_bool'].unique()


<BooleanArray>
[<NA>, False, True]
Length: 3, dtype: boolean

##### Cleaning `valenzuela_from_school['baranggay']`


In [10]:
bgy_list2 = ['assumption ville', 
            'balangkas', 
            'canumay', 
            'coloong', 
            'gen. t. de leon', 
            'karuhatan', 
            'lawa', 
            'lawang bato', 
            'lingunan', 
            'malinta', 
            'marulas', 
            'marulas', 
            'meycauayan', 
            'pinagbayanan',
            'punturin', 
            'quezon city', 
            'ugong', 
            'viente reales',
            'ciudad grande',
            'wala',
            'home',
            'bahay'
            ]

bgy_dict2 = {'ciudad grande':'meycauayan, bulacan',
             'lawa':'meycauayan, bulacan',
             'pag ibig st.':'lingunan',
             'brgy 176 bagong silang': 'caloocan',
             'tampoy2':'marulas',
             'meycauayan':'meycauayan, bulacan',
             'p.gregorio st.':'lingunan',
             'assumption ville':'lingunan',
             'sumapang matanda malolos':'malolos, bulacan',
             '688 t santaigo':'viente reales',
             'sto. niño biñan, laguna':'biñan, laguna',
             'san diego':'lingunan',
             'p.gregorio':'lingunan',
             'never been to other brgy. from school':'went home',
             'bahay':'went home',
             'home':'went home',
             'wala':'went home'
             }

valenzuela_from_school['baranggay'] = valenzuela_from_school['baranggay'].fillna('went home').apply(clean_text, args=[bgy_list2])
valenzuela_from_school['baranggay'].replace(bgy_dict2, inplace=True)

valenzuela_from_school['baranggay'].unique()

array(['balangkas', 'caloocan', 'punturin', 'ugong', 'rincon',
       'gen. t. de leon', 'marulas', 'lingunan', 'meycauayan, bulacan',
       'canumay', 'parada', 'viente reales', 'coloong', 'went home',
       'valenzuela', 'lawang bato', 'quezon city', 'malolos, bulacan',
       'karuhatan', 'pinagbayanan', 'biñan, laguna'], dtype=object)

##### Cleaning `valenzuela_from_school['location_after_school']`

In [11]:
after_dict = {'Deretso uwi na ':'directly went home',
              'bahay':'directly went home',
              'deretso uwi na ':'directly went home',
              'service kasi madami pa hinahatid bago ako':'inside school service',
              'waiting for other service mates':'inside school service',
              'playground':'mall/ restaurant/ commercial establishment/ place of recreation',
              'stores nearby':'mall/ restaurant/ commercial establishment/ place of recreation',
              'swimming practice':'mall/ restaurant/ commercial establishment/ place of recreation',
              'matgal masundo ng trykel kc traffic daw po':'inside school service',
              'palengke':'mall/ restaurant/ commercial establishment/ place of recreation',
              'tindahan':'mall/ restaurant/ commercial establishment/ place of recreation',
              'shop':'mall/ restaurant/ commercial establishment/ place of recreation'
              }

valenzuela_from_school['location_after_school'] = valenzuela_from_school["location_after_school"].str.lower().fillna('directly went home')
valenzuela_from_school['location_after_school'].replace(after_dict, inplace=True)
valenzuela_from_school['location_after_school'].unique()

array(['directly went home',
       'mall/ restaurant/ commercial establishment/ place of recreation',
       'inside school service', 'office of parent/guardian'], dtype=object)

##### Cleaning `valenzuela_from_school['companion']`

In [12]:
cmp_list2 = ['auntie',
            'service',
            'guardian',
            'grandmother',
            'grandfather',
            'brother',
            'sister',
            'grandparent',
            'father',
            'mother',
            'friends',
            'schoolmate',
            'classmate',
            'kaibigan'
            ]

cmp_cat2 = {'auntie':'adult relative',
            'brother':'sibling',
            'classmate':'schoolmate',
            'driver':'service',
            'friends':'schoolmate',
            'grandfather':'grandparent',
            'grandmother':'grandparent',
            'husband':'father',
            'kaibigan':'schoolmate',
            'kaklase':'schoolmate',
            'kamag anak':'adult relative',
            'kapit bahay sa street namin':'schoolmate',
            'kapwa mag aaral ko p0':'schoolmate',
            'lola':'grandparent',
            'kapatid':'sibling',
            'mama minsan si papa':'mother',
            'parents / relatives':'adult relative',
            'tita':'adult relative',
            'tito':'adult relative',
            'uncle tricycle driver':'service',
            'yaya':'guardian',
            'pinsan':'cousin',
            'wala':'none'
            }

valenzuela_from_school['companion'] = valenzuela_from_school['companion'].apply(clean_text, args=[cmp_list2]).astype('category')
valenzuela_from_school['companion'] = valenzuela_from_school['companion'].cat.add_categories('none').fillna('none')
valenzuela_from_school['companion'].replace(cmp_cat2, inplace=True)
valenzuela_from_school['companion'].cat.categories

Index(['adult relative', 'sibling', 'cousin', 'father', 'grandparent',
       'guardian', 'mother', 'schoolmate', 'service', 'none'],
      dtype='object')

In [13]:
pd.to_pickle(valenzuela_to_school,'./cleaned_data/valenzuela_to_school.pkl')
pd.to_pickle(valenzuela_from_school,'./cleaned_data/valenzuela_from_school.pkl')